In [1]:
import torch
import numpy as np
import os
import numpy as np
import torch
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split
from torch.utils.data import Dataset, DataLoader,RandomSampler
from torchvision import transforms
import tifffile
import glob
import torchsummary
from torch.utils.data import random_split
import tqdm
import models
from data import *
import torch.optim as optim


# Checking if CUDA is available
flag_cuda = torch.cuda.is_available()

if not flag_cuda:
    if torch.backends.mps.is_available():
        print('Using mps')
        device = 'mps'
    else:
        print('Using CPU')
        device = 'cpu'
else:
    print('Using GPU')
    device = 'GPU'

/Users/karthiksanka/opt/anaconda3/envs/deeplearning/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/karthiksanka/opt/anaconda3/envs/deeplearning/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Using mps


In [82]:
root_dir = 'Residential_density/NAIP(US_Dense)'
dataset = NAIP(root_dir, transform=preprocess)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
train_dataset, val_dataset = split_dataset(dataset = dataset, validation_fraction=0.2)
# train_loader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=128)
# val_loader = DataLoader(dataset=val_dataset, shuffle=False, batch_size=128)
rand_sampler = torch.utils.data.RandomSampler(train_dataset, num_samples=64, replacement=True)
rand_loader_naip = DataLoader(train_dataset,sampler=rand_sampler,batch_size=64)
root_dir = 'Residential_density/UC_merced'
dataset = UCmerced(root_dir, transform=preprocess)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
train_dataset, val_dataset = split_dataset(dataset = dataset, validation_fraction=0.2)
# train_loader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=128)
# val_loader = DataLoader(dataset=val_dataset, shuffle=False, batch_size=128)
rand_sampler = torch.utils.data.RandomSampler(train_dataset, num_samples=64, replacement=True)
rand_loader_ucm = DataLoader(train_dataset,sampler=rand_sampler,batch_size=64)


In [81]:
a,b,c = next(iter(rand_loader))

In [ ]:
for i in range(1000):
    a,b,c = next(iter(rand_loader))
print(i)

In [78]:
c.size()

torch.Size([1000])

In [7]:

model = models.get_baseline_clf().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),0.003)
model.train()
epochs = 40
for epoch in tqdm.tqdm(range(epochs)):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels,dom_label = data
        inputs,labels= inputs.to(device),labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 10:.3f}')
            running_loss = 0.0
        # print(f"epoch : {epoch} loss : {loss.item()}")
        # running_loss += loss.item()
        # print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / (epoch+1) :.3f}')

print('Finished Training')

/Users/karthiksanka/opt/anaconda3/envs/deeplearning/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/karthiksanka/opt/anaconda3/envs/deeplearning/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 40/40 [01:13<00:00,  1.83s/it]

Finished Training


In [3]:
model = models.get_baseline_clf(finetuned=True)

In [4]:
models.evaluate_model(model,dataloader,device=device)

Accuracy of the network on the validation images: 72 %


72.01834862385321

In [51]:
ae = models.autoencoder

In [56]:
#inp = torch.randn((3,224,224))
torchsummary.summary(ae.encoder,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [67]:
class VAE(nn.Module):
    def __init__(self, latent_dim,encoder,decoder):
        super(VAE, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = encoder

        self.decoder = decoder

    def encode(self, x):
        h = self.encoder(x)
        mu, logvar = torch.split(h, self.latent_dim, dim=1)
        return mu, logvar

    def decode(self, z):
        x = self.decoder(z)
        return x

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        z = mu + eps*std
        return z

    def forward(self, x,return_enc = False):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        if return_enc:
            return z
        x_hat = self.decode(z)
        return x_hat, mu, logvar


In [77]:
digit_cls = nn.Sequential(nn.Linear(1000,256),
                            nn.ReLU(),
                            nn.Linear(256,10))
domain_cls = nn.Sequential(nn.Linear(1000,256),
                            nn.ReLU(),
                            nn.Linear(256,2))

In [70]:
enc = models.encoder
enc.fc = nn.Linear(2048,2000)
dec = models.decoder
vae = VAE(1000,enc,dec)

In [68]:
torchsummary.summary(vae,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [72]:
inp = torch.randn((1,3,224,224))
outs = vae(inp,return_enc = True)
outs.shape

torch.Size([1, 1000])